In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from qiskit import BasicAer

from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import algorithm_globals, quantum_instance
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel, BaseKernel
from qiskit import Aer, QuantumCircuit

In [2]:
feature_df = pd.read_csv('feature_df_6.csv')
feature_df

,mean,variance,skewness,kurtosis,psd,HHT_Component_1,frequency
0,425.906361,4637.904896,-0.009724,-0.826606,6410.837739,0.003950,0
1,551.659794,2304.352609,-0.451809,-0.003270,3887.976974,0.002150,0
2,635.578799,2900.197315,0.970588,0.921783,2905.458353,0.000641,0
3,552.335149,22790.486041,0.377306,-1.223400,16129.607092,0.001005,0
4,501.169717,3458.247232,-0.936677,0.136579,2900.989626,0.001921,0
...,...,...,...,...,...,...,...
699,266.158515,193.060911,-0.283150,-0.635364,310.609248,0.006515,3
700,241.872231,103.425253,0.104029,-0.368583,188.273587,0.007861,3
701,256.513993,104.256258,0.088085,-0.253929,209.712606,0.009023,3
702,300.312222,188.100847,-0.573447,0.722081,195.639759,0.008149,3


In [3]:
# Feature Scaling

from sklearn.preprocessing import MinMaxScaler

scaled_df = feature_df.copy()

columns_to_standardize = scaled_df.columns[:-1]

# Create a subset DataFrame with the selected columns
df_to_standardize = scaled_df[columns_to_standardize]

scaler = MinMaxScaler()
df_standardized = pd.DataFrame(scaler.fit_transform(df_to_standardize))

# Update the original DataFrame with the standardized values
scaled_df[columns_to_standardize] = df_standardized

scaled_df

,mean,variance,skewness,kurtosis,psd,HHT_Component_1,frequency
0,0.490051,0.100536,0.686403,0.101319,0.071774,0.286590,0
1,0.674511,0.048798,0.556955,0.218854,0.042853,0.130745,0
2,0.797607,0.062008,0.973451,0.350910,0.031590,0.000000,0
3,0.675502,0.503004,0.799731,0.044674,0.183184,0.031565,0
4,0.600450,0.074381,0.414980,0.238818,0.031539,0.110863,0
...,...,...,...,...,...,...,...
699,0.255726,0.001987,0.606341,0.128619,0.001845,0.508802,3
700,0.220101,0.000000,0.719711,0.166704,0.000442,0.625406,3
701,0.241579,0.000018,0.715043,0.183071,0.000688,0.725989,3
702,0.305824,0.001877,0.521338,0.322401,0.000527,0.650360,3


In [4]:
from sklearn.model_selection import train_test_split

X = scaled_df.drop(columns=["frequency"]).values
y = scaled_df["frequency"].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, stratify=y)

In [5]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# Build a feature map
feature_map = ZZFeatureMap(feature_dimension=6, reps=2, entanglement="linear")

# Create a quantum kernel
qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [6]:
# Defining QSVC

qsvc = QSVC(quantum_kernel=qkernel)

In [7]:
# Model Training

import time
start = time.time()

qsvc.fit(X_train, y_train)

print(start - time.time())

-1508.3000888824463


In [9]:
eval_df = pd.DataFrame(columns=['predicted_label', 'true_label'])
eval_df['predicted_label'] = qsvc.predict(X_test)
eval_df['true_label'] = y_test

In [10]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, eval_df['predicted_label'].values)
accuracy

0.6028368794326241

In [11]:
# qsvc.save("trained_qsvc_6")

In [18]:
# Model Training

import time
start = time.time()

y_pred_train = qsvc.predict(X_train[:3])

print(time.time() - start)

10.019906044006348


In [19]:
X_train.shape

(563, 6)

In [22]:
loaded = QSVC.load("trained_qsvc_6")

# Model Training

import time
start = time.time()

_ = loaded.predict(X_train[:1])

print(time.time() - start)

3.3898229598999023


From https://qiskit.org/documentation/stable/0.24/tutorials/machine_learning/01_qsvm_classification.html

https://qiskit.org/ecosystem/machine-learning/stubs/qiskit_machine_learning.kernels.BaseKernel.html#qiskit_machine_learning.kernels.BaseKernel

https://qiskit.org/documentation/stable/0.24/tutorials/machine_learning/01_qsvm_classification.html

https://qiskit.org/ecosystem/machine-learning/tutorials/03_quantum_kernel.html

https://qiskit.org/ecosystem/machine-learning/tutorials/08_quantum_kernel_trainer.html

https://www.google.com/search?client=safari&rls=en&q=create+a+quantum+kernel+for+qsvc&ie=UTF-8&oe=UTF-8

https://qiskit.org/documentation/migration_guides/qi_migration.html